# Michaelis-Menten kinetics dynamics

### The Michaelis-Menten kinetics is given by the following system of (coupled) ordinary differential equations (ODEs):

$$
\dot{x}^{(\nu)}_i = -x^{(\nu)}_i + \sum_j w_{i,j} \cdot \frac{x^{(\nu)}_j}{1 + x^{(\nu)}_j}, \quad i, j \in \{1, \dots, N\}
$$

# TODOs
- Use Pandas' DataFrame instead of messy python lists
- "Colonial expansion" - Make the defects uneven (e.g. base_model W $->$ 10 Ws with p = 0.2 and 40 Ws with p = 0.8)
- "Colonial expansion" should decrease GCL (higher p $->$ lower GCL)

In [1]:
# Data manipulation libraries
import numpy as np
import pandas as pd

# Mathematical functions (ode solvers, spearman, PCA)
from scipy.integrate import solve_ivp
from scipy.stats import spearmanr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, scale

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

## TODO: Refactor to use pandas dataframes

from gcl_library import jackknife

$$
\dot{x}^{(\nu)}_i = -Bx^{(\nu)}_i + \sum_j w_{i,j} \cdot \frac{x^{(\nu)}_j}{1 + x^{(\nu)}_j}, \quad i, j \in \{1, \dots, N\}
$$

In [2]:
# Set up the system of ODEs
def ode_system(t, x, W, B, zero_mask):
    n = len(x)  # Get the number of elements in the state vector
    derivative_vector = np.zeros(n)  # Initialize the derivative vector (set zeros as default)
    for i in range(n):  # Loop over each element in the state vector
        # Compute the sum term for the i-th element according to the formula given in the paper
        sum_term = sum(W[i, j] * (x[j] / (1 + x[j])) for j in range(n) if j != i)
        derivative_vector[i] = -B * x[i] + sum_term
    # Set the derivative of the zeroed genes to 0
    for z in zero_mask:
        derivative_vector[z] = 0
    return derivative_vector

N represents # of genes in each cell  
num_of_cells (M) represents # of cells as well as # of defects matrices (defects of weight matrix w)
p represents the probability of the base weight matrix to defect (non-zero elements probability to change)  
A cohort is a group of cells with similar properties (in our case identical p value and the same base_model weight matrix)  
A cell is a network of genes  
A gene is a node in the network with its connection's represented by the relevant weight matrix W  
We have <num_of_cohorts> cohorts, in each cohort we have <num_of_cells> (previously called M) cells, all cells in this cohort have evolved according to their corresponding weight matrix

In [ ]:
# Initialize parameters
N = 200  # Number of genes in a cell
num_of_cells = 100  #100  # (M) Number of cells in each cohort = # of defects matrices
avg_deg = 3  # Average # of non-zero elements in the matrix W

decide = lambda x: np.random.random() < x
q = 2  # Affects the range of the random numbers generated
num_of_cohorts = 5
p = 0.2
step_size = 1.0 / (num_of_cohorts - 1) if num_of_cohorts > 1 else 0
p_ratios = [i * step_size for i in range(num_of_cohorts)]
num_of_cells_to_affect = [round(p_ratios[i] * num_of_cells) for i in range(num_of_cohorts)]
W = [[np.zeros((N, N)) for _ in range(num_of_cells)] for ___ in range(
    num_of_cohorts)]  # Initialize the weight matrix with zeros. W[cohort index][defect index / cell index][(row, column)] Perhaps there is a simpler more elegant way to do this with smaller dimension arrays.

# I.C is an array where each element represents a cohort with different p and each cohort has an array of cells and each cell has a vector of initial conditions for each gene in the cell. initial_conditions[cohort index][defect index / cell index][gene index]
initial_conditions = [[np.random.rand(N) for _ in range(num_of_cells)] for __ in range(num_of_cohorts)]

num_of_time_stamps = 1000  # Perhaps this is unnecessary for solve_ivp (t_eval is optional)
t_final = 20  # How will I know that 20 is enough to reach a steady state?
t = np.linspace(0, t_final, num_of_time_stamps)

In [ ]:
print(p_ratios)
print(num_of_cells_to_affect)

In [ ]:
for cohort_index in range(num_of_cohorts):
    cur_num_of_cells_to_affect = num_of_cells_to_affect[cohort_index]
    # For each cohort (group of <num_of_cells> cells), create a system of <num_of_cells> w defect matrices

    # Set the base weight matrix for the current cohort
    # TODO: this is weird since I defined avg_deg as the average degree but I use it as a probability distribution (actual average might be different) maybe change that in the future
    # TODO: try to make the base model the same for all cohorts later
    base_model = np.array([[np.random.uniform(0, q) if (gene_i != gene_j and decide(avg_deg / (N - 1))) else 0
                            for gene_j in range(N)] for gene_i in range(N)])

    # The first iteration is useless since p[0] = 0 by definition, and it's a simple copy of base_model. Maybe change that in the future
    # Create defects of the weight matrix
    # Too many indentations, maybe refactor this code
    for defect_index in range(num_of_cells):
        effective_p_value = p if defect_index <= cur_num_of_cells_to_affect else 0
        for gene_i in range(N):
            for gene_j in range(N):
                if base_model[gene_i, gene_j] != 0:
                    if decide(effective_p_value):
                        W[cohort_index][defect_index][gene_i, gene_j] = np.random.uniform(0, q)
                    else:
                        W[cohort_index][defect_index][gene_i, gene_j] = base_model[gene_i, gene_j]
                else:
                    W[cohort_index][defect_index][gene_i, gene_j] = 0

In [ ]:
results = [[np.zeros((num_of_time_stamps, N)) for _ in range(num_of_cells)] for __ in
           range(num_of_cohorts)]  # Initialize the results array
# results[cohort index][defect index][(row, column)] - rows represents timestamps, columns represent gene index

# Solve the ODEs for each cell in each cohort (num_of_cells X num_of_cohorts systems of ODE's (each system has N (num_of_genes) ode's)) and the corresponding initial conditions for the cells
B = 1  #np.random.uniform(0.8, 1.2)
for cohort_index in range(num_of_cohorts):
    for defect_index in range(num_of_cells):
        # Before computing the results, make 5 of the genes in each cohort in every cell 0 and stay 0. This will create "chain reactions" in the network when solving the ode system
        zero_mask = []
        num_of_zero_genes = 5
        for _ in range(num_of_zero_genes):
            gene_index = np.random.randint(0, N - 1)
            initial_conditions[cohort_index][defect_index][gene_index] = 0
            zero_mask.append(gene_index)
        results[cohort_index][defect_index] = \
            solve_ivp(ode_system, [t[0], t[-1]], initial_conditions[cohort_index][defect_index],
                      args=(W[cohort_index][defect_index], B, zero_mask), t_eval=t)[
                "y"].T  # Transpose the solution so that rows = timestamps and columns = gene index

In [ ]:
# # Plot the results
# for cohort_index in range(num_of_cohorts):
#     for defect_index in range(num_of_cells):
#         for gene in range(N):
#             plt.plot(t, results[cohort_index][defect_index][:, gene])
# plt.xlabel('Time')
# plt.ylabel('x value')
# plt.title('Michaelis-Menten kinetics')
# plt.show()

In [ ]:
steady_state = np.zeros((num_of_cohorts, N, num_of_cells))
# steady_state[cohort index, gene index, cell index]

for cohort_j in range(num_of_cohorts):
    for cell_index in range(num_of_cells):
        steady_state[cohort_j, :, cell_index] = results[cohort_j][cell_index][
            -1]  # Get the last row (final time) of the cell (the steady state)

### Calculate Negative Spearman Correlation

In [ ]:
negative_spearman_steady_state = [(1 - (spearmanr(steady_state[i])[0])) for i in range(num_of_cohorts)]
# spearman_steady_state[cohort index, cell_i, cell_j]

In [ ]:
# get the off diagonal elements and graph a distribution of them:
off_diagonal_elements = [[] for _ in range(num_of_cohorts)]
# off_diagonal_elements[cohort index][off-diagonal element index]. The off-diagonal element index doesn't really have a meaning it is just a way to store these off-diagonal elements.

# Store the off diagonal elements in an array of arrays (array for each cohort)
for cohort_index in range(num_of_cohorts):
    for cell_i in range(num_of_cells):
        for cell_j in range(num_of_cells):
            if cell_j > cell_i:
                off_diagonal_elements[cohort_index].append(negative_spearman_steady_state[cohort_index][cell_i][cell_j])

# Round to 2 decimal places to make the graph look normal
off_diagonal_elements = [np.round(element, decimals=2) for element in off_diagonal_elements]

In [ ]:
# Plot the off-diagonal elements distribution
for cohort_index in range(num_of_cohorts):
    plt.hist(off_diagonal_elements[cohort_index], alpha=0.5,
             label='cohort with {:.2f}% affected cells'.format(p_ratios[cohort_index]))

plt.title('Off-diagonal negative spearman matrix elements')  # (matrix is symmetric)
plt.xlabel('Negative Spearman value')
plt.ylabel('Frequency')

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
negative_spearman_steady_state[3].round()

In [ ]:
# Step 1: Flatten the results array (cohorts, cells, genes) into a 2D array (cells x genes)
# This way we can scale according to all the results
flattened_data = []
cohort_labels = []

# Flatten results and create cohort labels (p = 0, ..., p = 1)
for cohort_index in range(num_of_cohorts):
    for cell_index in range(num_of_cells):
        # Flatten gene data for each cell and add to flattened_data
        flattened_data.append(results[cohort_index][cell_index].flatten())
        cohort_labels.append(f'percentage of affected cells = {p_ratios[cohort_index]}')

# Convert to a DataFrame
flattened_data = np.array(flattened_data)
data_df = pd.DataFrame(flattened_data)

# Step 2: Scale the data across *all* cells
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_df)

# Step 3: Perform PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled_data)

pc1 = 'Principal Component 1 (accounts for {:.2f}% of the variation)'.format(pca.explained_variance_ratio_[0] * 100)
pc2 = 'Principal Component 2 (accounts for {:.2f}% of the variation)'.format(pca.explained_variance_ratio_[1] * 100)

# Convert PCA result to a DataFrame and add cohort information
pca_df = pd.DataFrame(data=pca_data, columns=[pc1, pc2])
pca_df['cohort'] = cohort_labels

# Step 4: Plot the PCA results
plt.figure(figsize=(10, 8))
sns.scatterplot(x=pc1, y=pc2, hue='cohort', data=pca_df, legend='full')
plt.title('PCA of All Cells group by Cohort')
plt.show()

#### PCA
check out this video: https://youtu.be/FgakZw6K1QQ

##### PC1
- Calculate the average measurement for gene 1, 2, 3, ..., N - get the "center" of the data
- Recalculate the data relative to center (the center is the origin).
- Find the best fit linear line that passes through the origin for the data by choosing the line that maximizes the sum of the distances of the points projected by each one of the points on to the line - this line is PC1.
- Normalize the line to unit length it is the eigen vector (also called singular vector) of PC1. the proportion of each gene are called "loading scores" (kind of like tan(theta) = y/x in 2d). The average of the sum of squared distances (of the points projected on to the PC1 line) is the eigen value for PC1. The square root of the sum of the squared distances is called the singular value for PC1

##### PC2
- The line that is perpendicular to PC1 (m_pc1 * m_pc2 = -1 for 2d, the number of PC's will be the MIN(variables (genes), samples(cells))) This line is the eigen vector for PC2. The loading scores are the projection of the normalized line on to the gene1 axis (x) and the projection of the normalized line on to the gene2 axis (y). The average of the sum of squared distances (of the points projected on to the PC2 line) is the eigen value for PC2.

PC1 accounts for eigen_value_pc1/(eigen_value_pc1 + eigen_value_pc2) of the total variation
PC2 accounts for eigen_value_pc2 / (eigen_value_pc1 + eigen_value_pc2) of the total variation

We may have many PCi's, but after we calculated the principal components we simply project them to the 2 most dominant pc's (the ones with the highest eigen value i.e. highest variation) NOTE: this will only be good if they account for most of the variation.
Use a scree plot for the PCi's

In [ ]:
print(len(W))  # num_of_cohorts. W is an array that each element represents a cohort with different p
print(len(W[
              num_of_cohorts - 1]))  # num_of_cells. An element in W is an array of 50 weight matrices (each corresponds to a cell)
print(
    len(W[num_of_cohorts - 1][num_of_cells - 1]))  # N (num of genes in each cell). Each weight matrix is an NxN matrix
print(W[num_of_cohorts - 1][num_of_cells - 1][(
    N - 1, N - 1)])  # A connection between the last gene with itself in the last cell in the last cohort (should be 0)

In [ ]:
# Calculate jackknife for every cohort's steady state
jackknife_results = []
for cohort_index in range(num_of_cohorts):
    jackknife_results.append(jackknife(steady_state[cohort_index], 50, 0.75, 50))

# Plot a violin plot of the jackknife results such that every violin represents a cohort
plt.figure(figsize=(10, 8))
sns.violinplot(data=jackknife_results, legend='full')
plt.title('Jackknife results for every cohort')
plt.show()

In [ ]:
for f in range(num_of_cells):
    if (all((W[0][0] == W[0][f]).flatten())):
        pass
    else:
        print("False")


In [ ]:
# steady_state[cohort index, gene index, cell index]
temp = steady_state[0].round(3)


In [ ]:
temp